In [1]:
D=[
    {'面包','牛奶'},
    {'面包','尿布','啤酒'},
    {'尿布','啤酒'}
]

In [3]:
D_length=len(D)

In [1]:
from itertools import combinations # 组合C

In [21]:
all_combs=set()
for d in D:
    for k in range(1,len(d)+1): # k[1，len(d)]
        houxuan={c for c in combinations(d,k)} # 在一个交易中的候选项集
        all_combs=all_combs.union(houxuan)

In [22]:
all_combs

{('啤酒',),
 ('啤酒', '面包'),
 ('尿布',),
 ('尿布', '啤酒'),
 ('尿布', '啤酒', '面包'),
 ('尿布', '面包'),
 ('牛奶',),
 ('牛奶', '面包'),
 ('面包',)}

In [2]:
U=['User%s'%(i+1) for i in range(15)]

In [18]:
s=['面包','啤酒','牛奶','酸奶','咖啡','巧克力','可乐','雪碧']

In [19]:
v=['Video%s'%(i+1) for i in range(8)]

In [20]:
from faker import Faker
fk = Faker(locale="zh-CN")

In [21]:
fk.date()

'1977-07-16'

In [22]:
fk.msisdn()

'1442132544931'

In [23]:
import random
dd=[]
for _ in range(40):
    row={}
    iD=fk.msisdn()
    user=random.choice(U)
    item=random.choice(s)
    dt=fk.date()
    row['TID']=iD
    row['User']=user
    row['Item']=item
    row['Date']=dt
    dd.append(row)

In [94]:
import random
dd=[]
for _ in range(100):
    row={}
    iD=fk.msisdn()
    user=random.choice(U)
    item=random.choice(v)
    dt=fk.date()
    row['TID']=iD
    row['User']=user
    row['Item']=item
    row['Duration']=random.randint(1,30)
    row['Date']=dt
    dd.append(row)

In [24]:
pd.DataFrame(dd).to_csv('shopdata.csv',index=0)

In [10]:
import pandas as pd
data=pd.read_csv('shopdata.csv')

In [50]:
data

,TID,User,Item,Date
0,1618362291349,User20,牛奶,1971-10-28
1,5308144049332,User3,咖啡,2010-08-20
2,2277587921067,User20,酸奶,2022-01-07
3,9381482551162,User7,面包,1979-05-10
4,5003217237895,User12,啤酒,1983-01-08
...,...,...,...,...
95,6628604106677,User6,雪碧,1991-06-07
96,3591955192204,User13,可乐,2017-01-18
97,6428584646289,User20,雪糕,1973-05-22
98,3523918602608,User10,可乐,1998-02-24


In [52]:
data_agg={}
for u,it in zip(data['User'],data['Item']):
    if u not in data_agg:
        data_agg[u]=[it]
    else:
        data_agg[u]+=[it]

In [56]:
T=[set(data_agg[v]) for v in data_agg]

In [89]:
def apriori(T,SUPmin=0.5,CONFmin=0.5):
    from itertools import combinations,permutations
    def get_candidates():
        candidate_itemset=set()
        for t in T:
            for k in range(1,len(t)+1):
                itemset=set(combinations(t,k))
                candidate_itemset=candidate_itemset.union(itemset)
        return candidate_itemset
    def get_support(X):
        X=set(X)
        support=0
        for t in T:
            if t>=X:
                support+=1/len(T)
        return round(support,2)
    def get_supports(Xs):
        supports=[]
        for X in Xs:
            supports.append(get_support(X))
        return supports
    
    candidates=get_candidates()
    supports=get_supports(candidates)
    
    print('\n',"*"*20,'候选子项集',"*"*20,'\n')
    for candidate,support in zip(candidates,supports):
        print(set(candidate),'支持度:',support)

    def get_freq_itemset(candidates,supports):
        print('\n',"*"*20,'频繁项集',"*"*20,'\n')
        freq_itemsets=[]
        for candidate,support in zip(candidates,supports):
            if support>SUPmin:
                print(set(candidate),'支持度:',support)
                freq_itemsets.append(candidate)
        return freq_itemsets
    
    
    freq_itemsets=get_freq_itemset(candidates,supports)
    

    
    def get_rules(freq_itemsets):
        strong_rule=[]
        pre_rules=list(permutations(freq_itemsets,2))
        rule_supports=[]
        for rule in pre_rules:
            support=0
            for t in T:
                front_itemset=set(rule[0])
                back_itemset=set(rule[1])
                if t>=front_itemset and t>=back_itemset and not front_itemset&back_itemset:
                    support+=1/len(T)
            if support>0:
                rule_supports.append([rule,support])

        print('\n',"*"*20,'关联规则',"*"*20,'\n')
        for rule,support in rule_supports:
            sup_X_Y=round(support,2)
            sup_X=get_support(rule[0])
            conf_X_Y=round(sup_X_Y/sup_X,2)
            print('%s=>%s'%(set(rule[0]),set(rule[1])),'支持度:',sup_X_Y,'可信度',conf_X_Y)
            if sup_X_Y>SUPmin and conf_X_Y>CONFmin: 
                print('强关联规则:','%s=>%s'%(set(rule[0]),set(rule[1])),'支持度:',sup_X_Y,'可信度',conf_X_Y)
                strong_rule.append([rule,sup_X_Y,conf_X_Y])
#         return rule_supports,strong_rule

    get_rules(freq_itemsets)

In [90]:
apriori(T,SUPmin=0.3,CONFmin=0.3)


 ******************** 候选子项集 ******************** 

{'雪碧', '啤酒'} 支持度: 0.1
{'咖啡', '巧克力'} 支持度: 0.15
{'面包', '咖啡', '雪碧', '酸奶'} 支持度: 0.05
{'雪碧', '牛奶', '雪糕', '巧克力'} 支持度: 0.1
{'可乐', '雪糕'} 支持度: 0.3
{'雪碧', '面包'} 支持度: 0.2
{'牛奶', '雪糕', '面包', '巧克力'} 支持度: 0.1
{'雪碧', '雪糕', '酸奶', '巧克力'} 支持度: 0.1
{'酸奶', '面包', '巧克力'} 支持度: 0.1
{'可乐', '啤酒', '雪糕'} 支持度: 0.05
{'雪糕', '酸奶'} 支持度: 0.35
{'啤酒', '牛奶', '咖啡', '巧克力'} 支持度: 0.05
{'酸奶', '雪糕', '面包'} 支持度: 0.15
{'可乐', '啤酒', '酸奶'} 支持度: 0.05
{'雪糕', '酸奶'} 支持度: 0.35
{'雪碧', '牛奶', '酸奶', '巧克力'} 支持度: 0.05
{'可乐', '牛奶', '雪碧', '巧克力'} 支持度: 0.05
{'啤酒', '牛奶'} 支持度: 0.2
{'啤酒', '牛奶', '咖啡', '酸奶', '雪碧'} 支持度: 0.05
{'雪碧', '雪糕'} 支持度: 0.25
{'啤酒', '牛奶', '咖啡', '酸奶'} 支持度: 0.05
{'可乐', '啤酒', '巧克力'} 支持度: 0.1
{'可乐', '雪糕', '巧克力'} 支持度: 0.15
{'啤酒', '巧克力'} 支持度: 0.15
{'啤酒', '咖啡', '酸奶', '雪碧', '巧克力'} 支持度: 0.05
{'酸奶', '咖啡', '巧克力'} 支持度: 0.15
{'咖啡', '酸奶', '巧克力'} 支持度: 0.15
{'可乐', '雪糕', '酸奶'} 支持度: 0.2
{'牛奶', '可乐', '雪糕', '雪碧', '巧克力'} 支持度: 0.05
{'雪碧', '牛奶'} 支持度: 0.25
{'可乐', '牛奶', '咖啡', '啤酒'} 支持度: 0.05
{'可乐', '面包', '